In [2]:
!pip install yfinance
import numpy as np
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

In [3]:
df1= pdr.get_data_yahoo('AAPL','2011-01-01','2021-10-31')
df1=df1[['Open','Close']]
df1=df1.rename(columns={'Open':'Open_AAPL','Close':'Close_AAPL'})

df2= pdr.get_data_yahoo('MSFT','2011-01-01','2021-10-31')
df2=df2[['Open','Close']]
df2=df2.rename(columns={'Open':'Open_MSFT','Close':'Close_MSFT'})

df = df1.merge(df2,on = 'Date')
df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open_AAPL,Close_AAPL,Open_MSFT,Close_MSFT
Date,,,,
2011-01-03,11.630000,11.770357,28.049999,27.980000
2011-01-04,11.872857,11.831786,27.940001,28.090000
2011-01-05,11.769643,11.928571,27.900000,28.000000
2011-01-06,11.954286,11.918929,28.040001,28.820000
2011-01-07,11.928214,12.004286,28.639999,28.600000
...,...,...,...,...
2021-10-25,148.679993,148.639999,309.359985,308.130005
2021-10-26,149.330002,149.320007,311.000000,310.109985
2021-10-27,149.360001,148.850006,316.000000,323.170013


In [4]:
two_years = df.loc['2011-01-01':'2012-12-31']
two_years = two_years[['Close_AAPL','Close_MSFT']]
two_years['Return_AAPL'] = two_years['Close_AAPL'].pct_change()
two_years['Return_MSFT'] = two_years['Close_MSFT'].pct_change()
two_years = two_years[['Return_AAPL', 'Return_MSFT']]
two_years = two_years.dropna()
two_years

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Return_AAPL,Return_MSFT
Date,,
2011-01-04,0.005219,0.003931
2011-01-05,0.008180,-0.003204
2011-01-06,-0.000808,0.029286
2011-01-07,0.007161,-0.007634
2011-01-10,0.018833,-0.013287
...,...,...
2012-12-24,0.001617,-0.014208
2012-12-26,-0.013784,-0.007391
2012-12-27,0.004016,0.003723


In [5]:
matrix_of_return_values = np.asmatrix(two_years)
matrix_of_return_values = matrix_of_return_values.T

no_of_asset_classes = 2
no_of_portfolios = 1000

collection_of_random_asset_weights = []
portfolio_means = []
portfolio_standard_deviations = []
portfolio_sharpe_ratios = []

def get_random_asset_weights(no_of_asset_classes):
    random_asset_weights = np.random.rand(no_of_asset_classes)
    random_asset_weights /= sum(random_asset_weights)
    random_asset_weights = np.asmatrix(random_asset_weights)
    return random_asset_weights

def get_portfolio_statistics(asset_weights, matrix_of_return_values):
    asset_means = np.asmatrix(np.mean(matrix_of_return_values, axis = 1))
    asset_covariances = np.asmatrix(np.cov(matrix_of_return_values))
    portfolio_mean = asset_weights * asset_means
    portfolio_mean = portfolio_mean.item()
    portfolio_standard_deviation = np.sqrt(asset_weights * asset_covariances * asset_weights.T)
    portfolio_standard_deviation = portfolio_standard_deviation.item()
    portfolio_sharpe_ratio = portfolio_mean / portfolio_standard_deviation
    return portfolio_mean, portfolio_standard_deviation, portfolio_sharpe_ratio

while len(collection_of_random_asset_weights) < no_of_portfolios:
    random_asset_weights = get_random_asset_weights(no_of_asset_classes)
    collection_of_random_asset_weights.append(random_asset_weights)
    #a,b,c = get_portfolio_statistics(random_asset_weights, matrix_of_return_values)
    portfolio_means.append(get_portfolio_statistics(random_asset_weights, matrix_of_return_values)[0])
    portfolio_standard_deviations.append(get_portfolio_statistics(random_asset_weights, matrix_of_return_values)[1])
    portfolio_sharpe_ratios.append(get_portfolio_statistics(random_asset_weights, matrix_of_return_values)[2])

index = portfolio_sharpe_ratios.index(max(portfolio_sharpe_ratios))
print(collection_of_random_asset_weights[index])


[[9.99982933e-01 1.70665895e-05]]


In [6]:
some_wighe = pd.DataFrame({'AAPL':[0.75],'MSFT':[0.25]},index= [two_years.last_valid_index()])
some_wighe

,AAPL,MSFT
2012-12-31,0.75,0.25


In [7]:
captial=10000
allocation =  captial *some_wighe
allocation

,AAPL,MSFT
2012-12-31,7500.0,2500.0


In [8]:


#no_of_shares['AAPL'] /=df['Close_AAPL'].loc[some_wighe.index[0]]
print(df['Close_AAPL'].loc[some_wighe.last_valid_index()])
print(df['Close_MSFT'].loc[some_wighe.last_valid_index()])

no_of_shares = allocation
no_of_shares['AAPL'].iloc[0] /= df['Close_AAPL'].loc[some_wighe.last_valid_index()] 
no_of_shares['MSFT'].iloc[0] /= df['Close_MSFT'].loc[some_wighe.last_valid_index()] 
no_of_shares
#no_of_shares = allocation / df[[f'Close_{sym}'for sym in some_wighe.columns]].loc[some_wighe.index[0]].values
#no_of_shares

19.006071090698242
26.709999084472656


,AAPL,MSFT
2012-12-31,394.610752,93.597907


In [9]:
eight_years = df.loc['2012-12-31':]
eight_years['Open'] = eight_years['Open_AAPL']*no_of_shares['AAPL'].iloc[0]+eight_years['Open_MSFT']*no_of_shares['MSFT'].iloc[0]

eight_years['Close'] = eight_years['Close_AAPL']*no_of_shares['AAPL'].iloc[0]+eight_years['Close_MSFT']*no_of_shares['MSFT'].iloc[0]
eight_years

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Open_AAPL,Close_AAPL,Open_MSFT,Close_MSFT,Open,Close
Date,,,,,,
2012-12-31,18.233213,19.006071,26.590000,26.709999,9683.790407,10000.000000
2013-01-02,19.779285,19.608213,27.250000,27.620001,10355.661647,10322.786098
2013-01-03,19.567142,19.360714,27.629999,27.250000,10307.514887,10190.488844
2013-01-04,19.177500,18.821428,27.270000,26.740000,10120.062557,9929.945971
2013-01-07,18.642857,18.710714,26.770000,26.690001,9862.287659,9881.577228
...,...,...,...,...,...,...
2021-10-25,148.679993,148.639999,309.359985,308.130005,87626.170692,87495.265314
2021-10-26,149.330002,149.320007,311.000000,310.109985,88036.173232,87948.925784
2021-10-27,149.360001,148.850006,316.000000,323.170013,88516.000605,88985.849537


In [17]:
import sklearn
from sklearn.linear_model import LogisticRegression


In [18]:
short_time = 5
long_time = 15
eight_years['Close_Short_Rolling']= eight_years['Close'].rolling(window=short_time).mean()

eight_years['Close_Long_Rolling']= eight_years['Close'].rolling(window=long_time).mean()

eight_years = eight_years.dropna()
eight_years



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Open_AAPL,Close_AAPL,Open_MSFT,Close_MSFT,Open,Close,Close_Short_Rolling,Close_Long_Rolling
Date,,,,,,,,
2013-02-11,17.017857,17.140356,27.650000,27.860001,9303.411267,9371.406526,9165.405574,9128.607557
2013-02-12,17.125357,16.710714,27.879999,27.879999,9367.359428,9203.737105,9200.874572,9098.521463
2013-02-13,16.686071,16.678928,27.930000,28.030001,9198.692737,9205.233850,9241.019180,9056.983375
2013-02-14,16.590000,16.663929,27.920000,28.040001,9159.845991,9200.250930,9250.651678,9074.659061
2013-02-15,16.744642,16.434286,28.040001,28.010000,9232.101255,9106.823384,9217.490359,9094.524321
...,...,...,...,...,...,...,...,...
2021-10-25,148.679993,148.639999,309.359985,308.130005,87626.170692,87495.265314,87680.815288,85365.721147
2021-10-26,149.330002,149.320007,311.000000,310.109985,88036.173232,87948.925784,87760.205037,85714.925810
2021-10-27,149.360001,148.850006,316.000000,323.170013,88516.000605,88985.849537,88022.868655,86082.701927


In [22]:
from sklearn import metrics
X = eight_years.copy()
y = np.where(X.Close.shift(-1)>= X.Close,1,-1)
index= int(0.7*len(X))
X_train= X.iloc[:index]
X_test= X.iloc[index:]
y_train= y[:index]
y_test= y[index:]
logistic = LogisticRegression()
logistic = logistic.fit(X_train,y_train)
prediction = logistic.predict(X_test)
#print(metrics.confusion_matrix(y_test,prediction))
print(metrics.confusion_matrix(y_test, prediction))
print(logistic.score(X_test,y_test))

[[ 45 249]
 [ 42 324]]
0.5590909090909091


In [26]:
result =  X_test.copy()
result['Close_Diff']= X_test.Close.shift(-1)- X_test.Close
result['Prediction']= prediction
result['True_Lable'] = y_test
result['Daily_Return']= result['Close_Diff']*result['Prediction']
result['Cumulative_Return']=result['Daily_Return'].cumsum()
result.dropna(inplace=True)
result

,Open_AAPL,Close_AAPL,Open_MSFT,Close_MSFT,Open,Close,Close_Short_Rolling,Close_Long_Rolling,Close_Diff,Prediction,True_Lable,Daily_Return,Cumulative_Return
Date,,,,,,,,,,,,,
2019-03-21,47.505001,48.772499,117.139999,120.220001,29710.042905,30498.492974,29647.448906,28551.409053,-695.261389,1,-1,-695.261389,-695.261389
2019-03-22,48.834999,47.762501,119.500000,117.050003,30455.765539,29803.231584,29766.060924,28685.368943,-170.792688,1,-1,-170.792688,-866.054078
2019-03-25,47.877499,47.185001,116.559998,117.660004,29802.747489,29632.438896,29781.951772,28803.839738,-168.974469,1,-1,-168.974469,-1035.028547
2019-03-26,47.915001,46.697498,118.620003,117.910004,30010.358469,29463.464427,29791.948735,28916.644561,59.035028,1,1,59.035028,-975.993519
2019-03-27,47.187500,47.117500,117.879997,116.769997,29654.015821,29522.499455,29784.025467,29039.715619,39.639180,1,1,39.639180,-936.354339
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-22,149.690002,148.690002,310.399994,309.160004,88122.074028,87611.402786,87500.143903,84959.679574,-116.137471,1,-1,-116.137471,69319.510839
2021-10-25,148.679993,148.639999,309.359985,308.130005,87626.170692,87495.265314,87680.815288,85365.721147,453.660469,1,1,453.660469,69773.171308
2021-10-26,149.330002,149.320007,311.000000,310.109985,88036.173232,87948.925784,87760.205037,85714.925810,1036.923753,1,1,1036.923753,70810.095062


In [36]:
print(7500.0/19.006071090698242)
print(2500.0/26.709999084472656)


394.6107517018903
93.59790661517944


In [18]:
data = np.array([['','Col1','Col2'],
                ['Row1',1,2],
                ['Row2',3,4]])
                
print(pd.DataFrame(data=data[1:,1:],
                  index=data[1:,0],
                  columns=data[0,1:]))

     Col1 Col2
Row1    1    2
Row2    3    4
